<a href="https://colab.research.google.com/github/ctandrewtran/andrewtran/blob/master/Project_1_Model_2_(6).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Winerschniztal Diet! 

By Andrew Tran, Josh Allen, Ryan Bilas, Matt Mester, and Elizabeth Gurry

In [ ]:
#Installing num2words for use
!pip install num2words

fRequirement already satisfied: num2words in /usr/local/lib/python3.6/dist-packages (0.5.10)


In [ ]:
# Importing Appropriate Programs
%matplotlib inline
from pylab import *

import shutil
import sys
import os.path

if not shutil.which("pyomo"):
    !pip install -q pyomo
    assert(shutil.which("pyomo"))

if not (shutil.which("glpsol") or os.path.isfile("glpsol")):
    if "google.colab" in sys.modules:
        !apt-get install -y -qq glpk-utils
    else:
        try:
            !conda install -c conda-forge ipopt
        except:
            pass

assert(shutil.which("glpsol") or os.path.isfile("glpsol"))

from pyomo.environ import *

SOLVER = 'glpk'
EXECUTABLE = '/usr/bin/glpsol'

In [ ]:
#importing pandas for use
import pandas as pd

#used later on to capture variable results from pyomo
import sys
import io

from num2words import num2words # Installed at the top

#data stored on uconn's github; query parama raw=true needed to retrieve data properly
url = 'https://github.uconn.edu/Ant16112/opim3803/blob/master/GP_data.csv?raw=true'

#Reading in the same csv data as Model 1
df = pd.read_csv(url)

#making sugar calorie column for later calculation
df['Sugar_Calories'] = df['Sugar'] * 4

#replacing food or drink with integer id to avoid data cleaning manually
#there were some issues with the text; num2words was fastest solution
df['food_drink_id'] = df.index
df['food_drink_id'] = df['food_drink_id'].apply(num2words)
df['food_drink_id']  = df['food_drink_id'].str.replace(" ", '')
df['food_drink_id']  = df['food_drink_id'].str.replace("and", '')
df['food_drink_id']  = df['food_drink_id'].str.replace("-", '')

In [ ]:
#Loop we used to automate the writing of the hard code for the constraint

constraint_list = list()
for x in df.values:
  hard_coded_constraint = "model." + str(x[8]) + "*" + str(x[2]) + " + "
  constraint_list.append(hard_coded_constraint)

for x in constraint_list:
    con = x.replace("'", "")
    print(con)
    break

model.zero*15 + 


In [ ]:
df.head() #Showing me the first 5 rows of the data set

,Food or Drink,Calories,Protein,Carb,Sodium,Sugar,Weight Watchers Points,Sugar_Calories,food_drink_id
0,"Original Chili Cheese Dog, standard bun",320,15,31,1410,4,9,16,zero
1,"Original Chili Dog, standard bun",270,12,31,1160,4,8,16,one
2,"Original Mustard Dog, standard bun",240,10,27,810,4,7,16,two
3,"Original Relish Dog, standard bun",250,10,30,870,6,7,24,three
4,"Original Kraut Dog, standard bun",240,10,28,1040,5,7,20,four


# The Diet: IIFYM

Called "if it fits your macros," it is a diet where you simply calculate the calories you burn per day and eat under that threshold alongside protein/fat macro requirements. Any food or drink goes!

## IIFYM Guidlines
* Calculate basal metabolic rate (BMR) adjusted with total daily energy expenditure (TDEE) and eat 10%-15% calories below that to achieve weightloss
* Protein intake is between .7-1 grams per lb of bodyweight
* Remaining calories filled by carbs/fats

https://www.healthline.com/nutrition/iifym-guide#meeting-your-macros 


## CDC Health Guidelines
Added this in to make sure the diet is more healthy
* Sodium must be less than or equal to 2300 mg
* Sugar must bes less than 10% of calories for the day

https://www.cdc.gov/salt/pdfs/sodium_dietary_guidelines.pdf

https://www.cdc.gov/nutrition/data-statistics/know-your-limit-for-added-sugars.html#:~:text=Americans%20should%20keep%20their%20intake,part%20of%20a%20healthy%20diet.&text=For%20example%2C%20in%20a%202%2C000,should%20come%20from%20added%20sugars. 


## Calculate BMR below using the Original Harris-Benedict Equation
https://www.k-state.edu/paccats/Contents/PA/PDF/Physical%20Activity%20and%20Controlling%20Weight.pdf
https://bmi-calories.com/bmr-calculator.html

In [ ]:
#gathering neccesary inputs for basal metabolic rate calculation
print("Are you male or female; input m for male and f for female")
gender = input()

print("Input height in inches")
height = input()
height = float(height)

print("Input age in years")
age = input()
age = float(age)

print("Input weight in lbs")
weight = input()
weight = float(weight)

def bmr_calc(gender, height, age, weight):
  if gender == 'm':
    bmr = 66.4730 + (13.7516 * (weight * .45)) + (5.0033 * (height * 2.54)) - (6.7550 * age)
    return bmr
  elif gender == 'f':
    bmr = 655.0955 + (9.5634 * (weight * .45)) + (1.8496 * (height * 2.54)) - (4.6756  * age)
    return bmr
  else:
    print('Wrong info entered, try again.')

bmr = bmr_calc(gender,height,age,weight)
print("Basal Metabolic Rate is: " + str(bmr))

Are you male or female; input m for male and f for female
f
Input height in inches
50
Input age in years
21
Input weight in lbs
120
Basal Metabolic Rate is: 1308.2307


## BMR Adjustment for activity level

We now adjust BMR by TDEE; essentially adjusting per our exercise level

Encode by inputting Sedentary, Lightly Active, etc as outlined below. it is case sensitive.

*   Sedentary (desk job, little excercise)
*   Lightly Active (light excercise/sports 1-3 days a week)
*   Moderately Active (moderate excercise/sports 6-7 days a week)
*   Very Active (hard excercise every day, or 2x a day excercise)
*   Extra Active (hard exercise 2+ times a day, or marathon training)

In [ ]:
print("How active are you? Input activity rate.")
activity_rate = input()
#Multipliers can be found in the K-State article
if activity_rate == 'Sedentary':
  adj_bmr = bmr * 1.2
  print("Adjusted BMR is now: " + str(adj_bmr))
elif activity_rate == 'Lightly Active':
  adj_bmr = bmr * 1.375
  print("Adjusted BMR is now: " + str(adj_bmr))
elif activity_rate == 'Moderately Active':
  adj_bmr = bmr * 1.55
  print("Adjusted BMR is now: " + str(adj_bmr))
elif activity_rate == 'Very Active':
  adj_bmr = bmr * 1.725
  print("Adjusted BMR is now: " + str(adj_bmr))
elif activity_rate == 'Extra Active':
  adj_bmr = bmr * 1.9
  print("Adjusted BMR is now: " + str(adj_bmr))
else:
  print('Wrong input. Try again.')

How active are you? Input activity rate.
Lightly Active
Adjusted BMR is now: 1798.8172125


## Other Constraint Calculations

Here are the other constraints that require calculations due to it being dependant on changing variables like weight or BMR.

We adjusted the Healthline article suggestion for protein by removing the "1 gram per lb of bodyweight" limit and instead only have the requirement to get >= .7 grams of protein per lb of body weight as protein is the most satiating thing to ingest, which aids weightloss.

https://academic.oup.com/ajcn/article/87/5/1558S/4650426

In [ ]:
#calculating out the bounds 

#calories lowered by ~12% to achieve weightloss
adj_bmr = adj_bmr * .875

#protein has to be this many grams per lb of body weight
protein_lb = weight * .7

#sugar has to be less than 10% of your daily calories
#sodium has to be less than 2300 mg
sugar_cal_b = bmr * .1
sodium_b = 2300


print("Protein lower bound: " + str(protein_lb))
print("Sugar calorie  bound is: " + str(sugar_cal_b))
print("Sodium lower bound is: " + str(sodium_b))
print("Adjusted BMR for Weightloss is: " + str(adj_bmr))

Protein lower bound: 84.0
Sugar calorie  bound is: 130.82307
Sodium lower bound is: 2300
Adjusted BMR for Weightloss is: 1373.642235


# Model 1
This contains our basic model which has the protein, sugar, calorie, and sodium constraints.

In [ ]:
from pyomo.environ import *

SOLVER = 'glpk'
EXECUTABLE = '/usr/bin/glpsol'

#initialize model
model = ConcreteModel()

#loop through and create variables
for x in df.values:
  name = x[8]
  model.add_component(name, Var(within=NonNegativeReals))

model.cal_const = Constraint(expr = model.zero*320 + 
model.one*270 + 
model.two*240 + 
model.three*250 + 
model.four*240 + 
model.five*250 + 
model.six*240 + 
model.seven*300 + 
model.eight*350 + 
model.nine*340 + 
model.ten*370 + 
model.eleven*460 + 
model.twelve*520 + 
model.thirteen*460 + 
model.fourteen*430 + 
model.fifteen*440 + 
model.sixteen*430 + 
model.seventeen*440 + 
model.eighteen*540 + 
model.nineteen*430 + 
model.twenty*440 + 
model.twentyone*490 + 
model.twentytwo*550 + 
model.twentythree*600 + 
model.twentyfour*370 + 
model.twentyfive*380 + 
model.twentysix*370 + 
model.twentyseven*380 + 
model.twentyeight*510 + 
model.twentynine*370 + 
model.thirty*380 + 
model.thirtyone*430 + 
model.thirtytwo*400 + 
model.thirtythree*450 + 
model.thirtyfour*600 + 
model.thirtyfive*540 + 
model.thirtysix*510 + 
model.thirtyseven*520 + 
model.thirtyeight*510 + 
model.thirtynine*520 + 
model.forty*620 + 
model.fortyone*510 + 
model.fortytwo*520 + 
model.fortythree*570 + 
model.fortyfour*620 + 
model.fortyfive*680 + 
model.fortysix*510 + 
model.fortyseven*610 + 
model.fortyeight*320 + 
model.fortynine*370 + 
model.fifty*640 + 
model.fiftyone*390 + 
model.fiftytwo*300 + 
model.fiftythree*470 + 
model.fiftyfour*350 + 
model.fiftyfive*570 + 
model.fiftysix*380 + 
model.fiftyseven*640 + 
model.fiftyeight*380 + 
model.fiftynine*400 + 
model.sixty*590 + 
model.sixtyone*510 + 
model.sixtytwo*350 + 
model.sixtythree*450 + 
model.sixtyfour*710 + 
model.sixtyfive*250 + 
model.sixtysix*320 + 
model.sixtyseven*280 + 
model.sixtyeight*470 + 
model.sixtynine*490 + 
model.seventy*410 + 
model.seventyone*290 + 
model.seventytwo*360 + 
model.seventythree*540 + 
model.seventyfour*1070 + 
model.seventyfive*150 + 
model.seventysix*60 + 
model.seventyseven*25 + 
model.seventyeight*300 + 
model.seventynine*430 + 
model.eighty*790 + 
model.eightyone*540 + 
model.eightytwo*370 + 
model.eightythree*210 + 
model.eightyfour*420 + 
model.eightyfive*490 + 
model.eightysix*590 + 
model.eightyseven*470 + 
model.eightyeight*370 + 
model.eightynine*330 + 
model.ninety*430 + 
model.ninetyone*390 + 
model.ninetytwo*490 + 
model.ninetythree*440 + 
model.ninetyfour*540 + 
model.ninetyfive*520 + 
model.ninetysix*620 + 
model.ninetyseven*300 + 
model.ninetyeight*400 + 
model.ninetynine*600 + 
model.onehundred*700 + 
model.onehundredone*640 + 
model.onehundredtwo*260 + 
model.onehundredthree*320 + 
model.onehundredfour*350 + 
model.onehundredfive*290 + 
model.onehundredsix*490 + 
model.onehundredseven*120 + 
model.onehundredeight*140 + 
model.onehundrednine*0 + 
model.onehundredten*0 + 
model.onehundredeleven*130 + 
model.onehundredtwelve*110 + 
model.onehundredthirteen*160 + 
model.onehundredfourteen*200 + 
model.onehundredfifteen*150 + 
model.onehundredsixteen*0 + 
model.onehundredseventeen*170 + 
model.onehundredeighteen*140 + 
model.onehundrednineteen*160 + 
model.onehundredtwenty*140 + 
model.onehundredtwentyone*140 + 
model.onehundredtwentytwo*150 + 
model.onehundredtwentythree*200 + 
model.onehundredtwentyfour*0 + 
model.onehundredtwentyfive*230 + 
model.onehundredtwentysix*190 + 
model.onehundredtwentyseven*210 + 
model.onehundredtwentyeight*180 + 
model.onehundredtwentynine*180 + 
model.onehundredthirty*200 + 
model.onehundredthirtyone*250 + 
model.onehundredthirtytwo*0 + 
model.onehundredthirtythree*280 + 
model.onehundredthirtyfour*230 + 
model.onehundredthirtyfive*270 + 
model.onehundredthirtysix*230 + 
model.onehundredthirtyseven*230 + 
model.onehundredthirtyeight*250 + 
model.onehundredthirtynine*320 + 
model.onehundredforty*310 + 
model.onehundredfortyone*370 + 
model.onehundredfortytwo*0 + 
model.onehundredfortythree*420 + 
model.onehundredfortyfour*350 + 
model.onehundredfortyfive*400 + 
model.onehundredfortysix*340 + 
model.onehundredfortyseven*340 + 
model.onehundredfortyeight*375 + 
model.onehundredfortynine*500 + 
model.onehundredfifty*0 + 
model.onehundredfiftyone*570 + 
model.onehundredfiftytwo*470 + 
model.onehundredfiftythree*530 + 
model.onehundredfiftyfour*450 + 
model.onehundredfiftyfive*450 + 
model.onehundredfiftysix*490 + 
model.onehundredfiftyseven*400 + 
model.onehundredfiftyeight*390 + 
model.onehundredfiftynine*370 + 
model.onehundredsixty*400 + 
model.onehundredsixtyone*370 + 
model.onehundredsixtytwo*250 + 
model.onehundredsixtythree*450 + 
model.onehundredsixtyfour*160 + 
model.onehundredsixtyfive*360 + 
model.onehundredsixtysix*820 + 
model.onehundredsixtyseven*650 + 
model.onehundredsixtyeight*650 + 
model.onehundredsixtynine*650 + 
model.onehundredseventy*630 + 
model.onehundredseventyone*630 + 
model.onehundredseventytwo*620 + 
model.onehundredseventythree*630 + 
model.onehundredseventyfour*940 + 
model.onehundredseventyfive*440 + 
model.onehundredseventysix*200 + 
model.onehundredseventyseven*180 + 
model.onehundredseventyeight*170 + 
model.onehundredseventynine*200  == adj_bmr)

model.proteinlb = Constraint(expr = model.zero*15 + 
model.one*12 + 
model.two*10 + 
model.three*10 + 
model.four*10 + 
model.five*10 + 
model.six*10 + 
model.seven*11 + 
model.eight*17 + 
model.nine*12 + 
model.ten*12 + 
model.eleven*19 + 
model.twelve*22 + 
model.thirteen*19 + 
model.fourteen*17 + 
model.fifteen*17 + 
model.sixteen*17 + 
model.seventeen*17 + 
model.eighteen*22 + 
model.nineteen*17 + 
model.twenty*17 + 
model.twentyone*18 + 
model.twentytwo*25 + 
model.twentythree*23 + 
model.twentyfour*13 + 
model.twentyfive*13 + 
model.twentysix*13 + 
model.twentyseven*13 + 
model.twentyeight*17 + 
model.twentynine*13 + 
model.thirty*13 + 
model.thirtyone*14 + 
model.thirtytwo*15 + 
model.thirtythree*18 + 
model.thirtyfour*21 + 
model.thirtyfive*18 + 
model.thirtysix*16 + 
model.thirtyseven*16 + 
model.thirtyeight*16 + 
model.thirtynine*16 + 
model.forty*21 + 
model.fortyone*16 + 
model.fortytwo*16 + 
model.fortythree*17 + 
model.fortyfour*23 + 
model.fortyfive*22 + 
model.fortysix*34 + 
model.fortyseven*39 + 
model.fortyeight*21 + 
model.fortynine*24 + 
model.fifty*43 + 
model.fiftyone*27 + 
model.fiftytwo*19 + 
model.fiftythree*33 + 
model.fiftyfour*22 + 
model.fiftyfive*38 + 
model.fiftysix*24 + 
model.fiftyseven*42 + 
model.fiftyeight*22 + 
model.fiftynine*23 + 
model.sixty*29 + 
model.sixtyone*24 + 
model.sixtytwo*11 + 
model.sixtythree*12 + 
model.sixtyfour*16 + 
model.sixtyfive*7 + 
model.sixtysix*8 + 
model.sixtyseven*12 + 
model.sixtyeight*25 + 
model.sixtynine*22 + 
model.seventy*12 + 
model.seventyone*16 + 
model.seventytwo*23 + 
model.seventythree*34 + 
model.seventyfour*69 + 
model.seventyfive*0 + 
model.seventysix*0 + 
model.seventyseven*0 + 
model.seventyeight*2 + 
model.seventynine*3 + 
model.eighty*6 + 
model.eightyone*12 + 
model.eightytwo*8 + 
model.eightythree*6 + 
model.eightyfour*13 + 
model.eightyfive*24 + 
model.eightysix*25 + 
model.eightyseven*25 + 
model.eightyeight*17 + 
model.eightynine*10 + 
model.ninety*11 + 
model.ninetyone*16 + 
model.ninetytwo*17 + 
model.ninetythree*18 + 
model.ninetyfour*19 + 
model.ninetyfive*19 + 
model.ninetysix*20 + 
model.ninetyseven*16 + 
model.ninetyeight*17 + 
model.ninetynine*20 + 
model.onehundred*21 + 
model.onehundredone*24 + 
model.onehundredtwo*6 + 
model.onehundredthree*12 + 
model.onehundredfour*8 + 
model.onehundredfive*1 + 
model.onehundredsix*6 + 
model.onehundredseven*0 + 
model.onehundredeight*0 + 
model.onehundrednine*0 + 
model.onehundredten*0 + 
model.onehundredeleven*10 + 
model.onehundredtwelve*9 + 
model.onehundredthirteen*8 + 
model.onehundredfourteen*11 + 
model.onehundredfifteen*0 + 
model.onehundredsixteen*0 + 
model.onehundredseventeen*0 + 
model.onehundredeighteen*0 + 
model.onehundrednineteen*0 + 
model.onehundredtwenty*0 + 
model.onehundredtwentyone*0 + 
model.onehundredtwentytwo*0 + 
model.onehundredtwentythree*0 + 
model.onehundredtwentyfour*0 + 
model.onehundredtwentyfive*0 + 
model.onehundredtwentysix*0 + 
model.onehundredtwentyseven*0 + 
model.onehundredtwentyeight*0 + 
model.onehundredtwentynine*0 + 
model.onehundredthirty*0 + 
model.onehundredthirtyone*0 + 
model.onehundredthirtytwo*0 + 
model.onehundredthirtythree*0 + 
model.onehundredthirtyfour*0 + 
model.onehundredthirtyfive*0 + 
model.onehundredthirtysix*0 + 
model.onehundredthirtyseven*0 + 
model.onehundredthirtyeight*0 + 
model.onehundredthirtynine*0 + 
model.onehundredforty*0 + 
model.onehundredfortyone*0 + 
model.onehundredfortytwo*0 + 
model.onehundredfortythree*0 + 
model.onehundredfortyfour*0 + 
model.onehundredfortyfive*0 + 
model.onehundredfortysix*0 + 
model.onehundredfortyseven*0 + 
model.onehundredfortyeight*0 + 
model.onehundredfortynine*0 + 
model.onehundredfifty*0 + 
model.onehundredfiftyone*0 + 
model.onehundredfiftytwo*0 + 
model.onehundredfiftythree*0 + 
model.onehundredfiftyfour*0 + 
model.onehundredfiftyfive*0 + 
model.onehundredfiftysix*0 + 
model.onehundredfiftyseven*7 + 
model.onehundredfiftyeight*7 + 
model.onehundredfiftynine*7 + 
model.onehundredsixty*7 + 
model.onehundredsixtyone*7 + 
model.onehundredsixtytwo*5 + 
model.onehundredsixtythree*6 + 
model.onehundredsixtyfour*3 + 
model.onehundredsixtyfive*4 + 
model.onehundredsixtysix*13 + 
model.onehundredsixtyseven*13 + 
model.onehundredsixtyeight*13 + 
model.onehundredsixtynine*13 + 
model.onehundredseventy*13 + 
model.onehundredseventyone*13 + 
model.onehundredseventytwo*13 + 
model.onehundredseventythree*14 + 
model.onehundredseventyfour*14 + 
model.onehundredseventyfive*7 + 
model.onehundredseventysix*3 + 
model.onehundredseventyseven*3 + 
model.onehundredseventyeight*3 + 
model.onehundredseventynine*3 >= protein_lb)

model.sugar = Constraint(expr = model.zero*4 + 
model.one*4 + 
model.two*4 + 
model.three*6 + 
model.four*5 + 
model.five*6 + 
model.six*4 + 
model.seven*14 + 
model.eight*5 + 
model.nine*4 + 
model.ten*6 + 
model.eleven*4 + 
model.twelve*5 + 
model.thirteen*4 + 
model.fourteen*4 + 
model.fifteen*6 + 
model.sixteen*4 + 
model.seventeen*6 + 
model.eighteen*9 + 
model.nineteen*4 + 
model.twenty*7 + 
model.twentyone*14 + 
model.twentytwo*5 + 
model.twentythree*5 + 
model.twentyfour*7 + 
model.twentyfive*9 + 
model.twentysix*8 + 
model.twentyseven*9 + 
model.twentyeight*12 + 
model.twentynine*7 + 
model.thirty*10 + 
model.thirtyone*17 + 
model.thirtytwo*7 + 
model.thirtythree*7 + 
model.thirtyfour*9 + 
model.thirtyfive*8 + 
model.thirtysix*8 + 
model.thirtyseven*10 + 
model.thirtyeight*8 + 
model.thirtynine*10 + 
model.forty*13 + 
model.fortyone*8 + 
model.fortytwo*11 + 
model.fortythree*18 + 
model.fortyfour*9 + 
model.fortyfive*9 + 
model.fortysix*6 + 
model.fortyseven*6 + 
model.fortyeight*3 + 
model.fortynine*3 + 
model.fifty*4 + 
model.fiftyone*3 + 
model.fiftytwo*5 + 
model.fiftythree*5 + 
model.fiftyfour*5 + 
model.fiftyfive*5 + 
model.fiftysix*5 + 
model.fiftyseven*5 + 
model.fiftyeight*6 + 
model.fiftynine*4 + 
model.sixty*6 + 
model.sixtyone*3 + 
model.sixtytwo*5 + 
model.sixtythree*1 + 
model.sixtyfour*9 + 
model.sixtyfive*1 + 
model.sixtysix*1 + 
model.sixtyseven*6 + 
model.sixtyeight*2 + 
model.sixtynine*4 + 
model.seventy*1 + 
model.seventyone*2 + 
model.seventytwo*1 + 
model.seventythree*2 + 
model.seventyfour*4 + 
model.seventyfive*1 + 
model.seventysix*12 + 
model.seventyseven*2 + 
model.seventyeight*0 + 
model.seventynine*0 + 
model.eighty*18 + 
model.eightyone*1 + 
model.eightytwo*1 + 
model.eightythree*3 + 
model.eightyfour*7 + 
model.eightyfive*3 + 
model.eightysix*3 + 
model.eightyseven*2 + 
model.eightyeight*1 + 
model.eightynine*4 + 
model.ninety*4 + 
model.ninetyone*4 + 
model.ninetytwo*4 + 
model.ninetythree*4 + 
model.ninetyfour*4 + 
model.ninetyfive*8 + 
model.ninetysix*8 + 
model.ninetyseven*1 + 
model.ninetyeight*1 + 
model.ninetynine*1 + 
model.onehundred*1 + 
model.onehundredone*4 + 
model.onehundredtwo*4 + 
model.onehundredthree*4 + 
model.onehundredfour*4 + 
model.onehundredfive*0 + 
model.onehundredsix*11 + 
model.onehundredseven*21 + 
model.onehundredeight*28 + 
model.onehundrednine*0 + 
model.onehundredten*0 + 
model.onehundredeleven*13 + 
model.onehundredtwelve*12 + 
model.onehundredthirteen*25 + 
model.onehundredfourteen*33 + 
model.onehundredfifteen*41 + 
model.onehundredsixteen*0 + 
model.onehundredseventeen*46 + 
model.onehundredeighteen*39 + 
model.onehundrednineteen*43 + 
model.onehundredtwenty*39 + 
model.onehundredtwentyone*35 + 
model.onehundredtwentytwo*41 + 
model.onehundredtwentythree*55 + 
model.onehundredtwentyfour*0 + 
model.onehundredtwentyfive*61 + 
model.onehundredtwentysix*52 + 
model.onehundredtwentyseven*57 + 
model.onehundredtwentyeight*52 + 
model.onehundredtwentynine*45 + 
model.onehundredthirty*54 + 
model.onehundredthirtyone*68 + 
model.onehundredthirtytwo*0 + 
model.onehundredthirtythree*77 + 
model.onehundredthirtyfour*65 + 
model.onehundredthirtyfive*72 + 
model.onehundredthirtysix*65 + 
model.onehundredthirtyseven*58 + 
model.onehundredthirtyeight*68 + 
model.onehundredthirtynine*84 + 
model.onehundredforty*81 + 
model.onehundredfortyone*103 + 
model.onehundredfortytwo*0 + 
model.onehundredfortythree*115 + 
model.onehundredfortyfour*97 + 
model.onehundredfortyfive*107 + 
model.onehundredfortysix*97 + 
model.onehundredfortyseven*86 + 
model.onehundredfortyeight*101 + 
model.onehundredfortynine*137 + 
model.onehundredfifty*0 + 
model.onehundredfiftyone*153 + 
model.onehundredfiftytwo*130 + 
model.onehundredfiftythree*143 + 
model.onehundredfiftyfour*130 + 
model.onehundredfiftyfive*115 + 
model.onehundredfiftysix*133 + 
model.onehundredfiftyseven*50 + 
model.onehundredfiftyeight*51 + 
model.onehundredfiftynine*48 + 
model.onehundredsixty*53 + 
model.onehundredsixtyone*49 + 
model.onehundredsixtytwo*30 + 
model.onehundredsixtythree*37 + 
model.onehundredsixtyfour*18 + 
model.onehundredsixtyfive*25 + 
model.onehundredsixtysix*122 + 
model.onehundredsixtyseven*91 + 
model.onehundredsixtyeight*91 + 
model.onehundredsixtynine*92 + 
model.onehundredseventy*76 + 
model.onehundredseventyone*80 + 
model.onehundredseventytwo*78 + 
model.onehundredseventythree*78 + 
model.onehundredseventyfour*87 + 
model.onehundredseventyfive*73 + 
model.onehundredseventysix*26 + 
model.onehundredseventyseven*28 + 
model.onehundredseventyeight*25 + 
model.onehundredseventynine*30 <= sugar_cal_b)

model.sodium = Constraint(expr = model.zero*1410 + 
model.one*1160 + 
model.two*810 + 
model.three*870 + 
model.four*1040 + 
model.five*1230 + 
model.six*760 + 
model.seven*2060 + 
model.eight*1670 + 
model.nine*880 + 
model.ten*970 + 
model.eleven*1720 + 
model.twelve*1440 + 
model.thirteen*1350 + 
model.fourteen*1000 + 
model.fifteen*1060 + 
model.sixteen*1230 + 
model.seventeen*1420 + 
model.eighteen*1310 + 
model.nineteen*950 + 
model.twenty*1100 + 
model.twentyone*2250 + 
model.twentytwo*1720 + 
model.twentythree*1800 + 
model.twentyfour*880 + 
model.twentyfive*930 + 
model.twentysix*1100 + 
model.twentyseven*1300 + 
model.twentyeight*1170 + 
model.twentynine*830 + 
model.thirty*970 + 
model.thirtyone*2120 + 
model.thirtytwo*1220 + 
model.thirtythree*1480 + 
model.thirtyfour*1420 + 
model.thirtyfive*1330 + 
model.thirtysix*990 + 
model.thirtyseven*1040 + 
model.thirtyeight*1210 + 
model.thirtynine*1410 + 
model.forty*1300 + 
model.fortyone*940 + 
model.fortytwo*1080 + 
model.fortythree*2230 + 
model.fortyfour*1850 + 
model.fortyfive*1780 + 
model.fortysix*870 + 
model.fortyseven*1380 + 
model.fortyeight*870 + 
model.fortynine*1130 + 
model.fifty*1940 + 
model.fiftyone*980 + 
model.fiftytwo*650 + 
model.fiftythree*810 + 
model.fiftyfour*910 + 
model.fiftyfive*1320 + 
model.fiftysix*1020 + 
model.fiftyseven*1550 + 
model.fiftyeight*970 + 
model.fiftynine*1480 + 
model.sixty*1810 + 
model.sixtyone*1400 + 
model.sixtytwo*640 + 
model.sixtythree*860 + 
model.sixtyfour*1820 + 
model.sixtyfive*490 + 
model.sixtysix*540 + 
model.sixtyseven*1180 + 
model.sixtyeight*1880 + 
model.sixtynine*1870 + 
model.seventy*1200 + 
model.seventyone*690 + 
model.seventytwo*1100 + 
model.seventythree*1640 + 
model.seventyfour*3290 + 
model.seventyfive*140 + 
model.seventysix*220 + 
model.seventyseven*1590 + 
model.seventyeight*400 + 
model.seventynine*630 + 
model.eighty*3600 + 
model.eightyone*1380 + 
model.eightytwo*800 + 
model.eightythree*670 + 
model.eightyfour*1340 + 
model.eightyfive*1380 + 
model.eightysix*1500 + 
model.eightyseven*1720 + 
model.eightyeight*1180 + 
model.eightynine*1120 + 
model.ninety*1240 + 
model.ninetyone*1270 + 
model.ninetytwo*1390 + 
model.ninetythree*1520 + 
model.ninetyfour*1640 + 
model.ninetyfive*990 + 
model.ninetysix*1110 + 
model.ninetyseven*830 + 
model.ninetyeight*950 + 
model.ninetynine*960 + 
model.onehundred*1080 + 
model.onehundredone*1820 + 
model.onehundredtwo*890 + 
model.onehundredthree*1040 + 
model.onehundredfour*1180 + 
model.onehundredfive*240 + 
model.onehundredsix*460 + 
model.onehundredseven*25 + 
model.onehundredeight*0 + 
model.onehundrednine*0 + 
model.onehundredten*0 + 
model.onehundredeleven*130 + 
model.onehundredtwelve*130 + 
model.onehundredthirteen*210 + 
model.onehundredfourteen*230 + 
model.onehundredfifteen*35 + 
model.onehundredsixteen*35 + 
model.onehundredseventeen*70 + 
model.onehundredeighteen*35 + 
model.onehundrednineteen*65 + 
model.onehundredtwenty*45 + 
model.onehundredtwentyone*75 + 
model.onehundredtwentytwo*160 + 
model.onehundredtwentythree*45 + 
model.onehundredtwentyfour*45 + 
model.onehundredtwentyfive*95 + 
model.onehundredtwentysix*45 + 
model.onehundredtwentyseven*85 + 
model.onehundredtwentyeight*60 + 
model.onehundredtwentynine*100 + 
model.onehundredthirty*210 + 
model.onehundredthirtyone*60 + 
model.onehundredthirtytwo*60 + 
model.onehundredthirtythree*115 + 
model.onehundredthirtyfour*60 + 
model.onehundredthirtyfive*110 + 
model.onehundredthirtysix*75 + 
model.onehundredthirtyseven*125 + 
model.onehundredthirtyeight*260 + 
model.onehundredthirtynine*250 + 
model.onehundredforty*260 + 
model.onehundredfortyone*90 + 
model.onehundredfortytwo*90 + 
model.onehundredfortythree*170 + 
model.onehundredfortyfour*85 + 
model.onehundredfortyfive*160 + 
model.onehundredfortysix*110 + 
model.onehundredfortyseven*190 + 
model.onehundredfortyeight*390 + 
model.onehundredfortynine*115 + 
model.onehundredfifty*115 + 
model.onehundredfiftyone*230 + 
model.onehundredfiftytwo*115 + 
model.onehundredfiftythree*220 + 
model.onehundredfiftyfour*150 + 
model.onehundredfiftyfive*250 + 
model.onehundredfiftysix*520 + 
model.onehundredfiftyseven*250 + 
model.onehundredfiftyeight*250 + 
model.onehundredfiftynine*220 + 
model.onehundredsixty*250 + 
model.onehundredsixtyone*230 + 
model.onehundredsixtytwo*180 + 
model.onehundredsixtythree*200 + 
model.onehundredsixtyfour*115 + 
model.onehundredsixtyfive*135 + 
model.onehundredsixtysix*340 + 
model.onehundredsixtyseven*440 + 
model.onehundredsixtyeight*480 + 
model.onehundredsixtynine*440 + 
model.onehundredseventy*410 + 
model.onehundredseventyone*420 + 
model.onehundredseventytwo*440 + 
model.onehundredseventythree*460 + 
model.onehundredseventyfour*540 + 
model.onehundredseventyfive*290 + 
model.onehundredseventysix*130 + 
model.onehundredseventyseven*135 + 
model.onehundredseventyeight*105 + 
model.onehundredseventynine*135 <= sodium_b)

In [ ]:
#creating objective function to minimize weight watchers points
model.points = Objective(
    expr = model.zero*9 + 
model.one*8 + 
model.two*7 + 
model.three*7 + 
model.four*7 + 
model.five*7 + 
model.six*7 + 
model.seven*8 + 
model.eight*10 + 
model.nine*9 + 
model.ten*10 + 
model.eleven*13 + 
model.twelve*14 + 
model.thirteen*12 + 
model.fourteen*12 + 
model.fifteen*12 + 
model.sixteen*12 + 
model.seventeen*12 + 
model.eighteen*15 + 
model.nineteen*12 + 
model.twenty*12 + 
model.twentyone*14 + 
model.twentytwo*15 + 
model.twentythree*17 + 
model.twentyfour*10 + 
model.twentyfive*11 + 
model.twentysix*10 + 
model.twentyseven*11 + 
model.twentyeight*14 + 
model.twentynine*10 + 
model.thirty*11 + 
model.thirtyone*12 + 
model.thirtytwo*11 + 
model.thirtythree*13 + 
model.thirtyfour*16 + 
model.thirtyfive*15 + 
model.thirtysix*14 + 
model.thirtyseven*14 + 
model.thirtyeight*14 + 
model.thirtynine*14 + 
model.forty*17 + 
model.fortyone*14 + 
model.fortytwo*14 + 
model.fortythree*16 + 
model.fortyfour*17 + 
model.fortyfive*19 + 
model.fortysix*15 + 
model.fortyseven*18 + 
model.fortyeight*9 + 
model.fortynine*10 + 
model.fifty*19 + 
model.fiftyone*10 + 
model.fiftytwo*8 + 
model.fiftythree*14 + 
model.fiftyfour*9 + 
model.fiftyfive*17 + 
model.fiftysix*10 + 
model.fiftyseven*19 + 
model.fiftyeight*10 + 
model.fiftynine*10 + 
model.sixty*16 + 
model.sixtyone*14 + 
model.sixtytwo*10 + 
model.sixtythree*12 + 
model.sixtyfour*20 + 
model.sixtyfive*7 + 
model.sixtysix*9 + 
model.sixtyseven*7 + 
model.sixtyeight*12 + 
model.sixtynine*14 + 
model.seventy*11 + 
model.seventyone*8 + 
model.seventytwo*10 + 
model.seventythree*14 + 
model.seventyfour*29 + 
model.seventyfive*4 + 
model.seventysix*1 + 
model.seventyseven*1 + 
model.seventyeight*9 + 
model.seventynine*12 + 
model.eighty*22 + 
model.eightyone*15 + 
model.eightytwo*11 + 
model.eightythree*6 + 
model.eightyfour*11 + 
model.eightyfive*13 + 
model.eightysix*16 + 
model.eightyseven*12 + 
model.eightyeight*10 + 
model.eightynine*9 + 
model.ninety*12 + 
model.ninetyone*11 + 
model.ninetytwo*14 + 
model.ninetythree*12 + 
model.ninetyfour*15 + 
model.ninetyfive*14 + 
model.ninetysix*17 + 
model.ninetyseven*8 + 
model.ninetyeight*11 + 
model.ninetynine*17 + 
model.onehundred*19 + 
model.onehundredone*18 + 
model.onehundredtwo*7 + 
model.onehundredthree*9 + 
model.onehundredfour*10 + 
model.onehundredfive*8 + 
model.onehundredsix*13 + 
model.onehundredseven*3 + 
model.onehundredeight*4 + 
model.onehundrednine*0 + 
model.onehundredten*0 + 
model.onehundredeleven*4 + 
model.onehundredtwelve*3 + 
model.onehundredthirteen*4 + 
model.onehundredfourteen*5 + 
model.onehundredfifteen*4 + 
model.onehundredsixteen*0 + 
model.onehundredseventeen*5 + 
model.onehundredeighteen*4 + 
model.onehundrednineteen*5 + 
model.onehundredtwenty*4 + 
model.onehundredtwentyone*4 + 
model.onehundredtwentytwo*4 + 
model.onehundredtwentythree*6 + 
model.onehundredtwentyfour*0 + 
model.onehundredtwentyfive*7 + 
model.onehundredtwentysix*6 + 
model.onehundredtwentyseven*6 + 
model.onehundredtwentyeight*6 + 
model.onehundredtwentynine*5 + 
model.onehundredthirty*6 + 
model.onehundredthirtyone*7 + 
model.onehundredthirtytwo*0 + 
model.onehundredthirtythree*8 + 
model.onehundredthirtyfour*7 + 
model.onehundredthirtyfive*8 + 
model.onehundredthirtysix*7 + 
model.onehundredthirtyseven*6 + 
model.onehundredthirtyeight*7 + 
model.onehundredthirtynine*9 + 
model.onehundredforty*9 + 
model.onehundredfortyone*11 + 
model.onehundredfortytwo*0 + 
model.onehundredfortythree*12 + 
model.onehundredfortyfour*11 + 
model.onehundredfortyfive*12 + 
model.onehundredfortysix*11 + 
model.onehundredfortyseven*9 + 
model.onehundredfortyeight*11 + 
model.onehundredfortynine*15 + 
model.onehundredfifty*0 + 
model.onehundredfiftyone*17 + 
model.onehundredfiftytwo*14 + 
model.onehundredfiftythree*16 + 
model.onehundredfiftyfour*14 + 
model.onehundredfiftyfive*12 + 
model.onehundredfiftysix*14 + 
model.onehundredfiftyseven*12 + 
model.onehundredfiftyeight*11 + 
model.onehundredfiftynine*11 + 
model.onehundredsixty*11 + 
model.onehundredsixtyone*11 + 
model.onehundredsixtytwo*7 + 
model.onehundredsixtythree*13 + 
model.onehundredsixtyfour*5 + 
model.onehundredsixtyfive*10 + 
model.onehundredsixtysix*24 + 
model.onehundredsixtyseven*19 + 
model.onehundredsixtyeight*19 + 
model.onehundredsixtynine*19 + 
model.onehundredseventy*18 + 
model.onehundredseventyone*18 + 
model.onehundredseventytwo*18 + 
model.onehundredseventythree*19 + 
model.onehundredseventyfour*28 + 
model.onehundredseventyfive*13 + 
model.onehundredseventysix*6 + 
model.onehundredseventyseven*5 + 
model.onehundredseventyeight*5 + 
model.onehundredseventynine*6, sense=minimize
    )

## Model 1 Results

Here, we output the models results and the weight watchers points. It is common to NOT get a result as the model is unable to work; we have a strict sodium limit as per the CDC guidelines and the menu is high in salt. Thus, higher calorie diets typically can't work.

In [ ]:
#printing model results and weight watchers points
SolverFactory(SOLVER, executable=EXECUTABLE).solve(model).write(num=1)
print(" ")
print("**       Weight Watchers Points: " + str(model.points()) + "       **")

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 34.9825727668861
  Upper bound: 34.9825727668861
  Number of objectives: 1
  Number of constraints: 5
  Number of variables: 179
  Number of nonzeros: 652
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.013035297393798828
# ----------------------------------------------------------
#   Solution Information
# ----------

In [ ]:
# Save the model 
model.write("/content/model.lp", io_options={'symbolic_solver_labels': True})

# Generate the file "sensit.sen", this contains the report we want to see
!/usr/bin/glpsol -m /content/model.lp --lp --ranges sensit.sen

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 -m /content/model.lp --lp --ranges sensit.sen
Reading problem data from '/content/model.lp'...
5 rows, 179 columns, 652 non-zeros
1026 lines were read
GLPK Simplex Optimizer, v4.65
5 rows, 179 columns, 652 non-zeros
Preprocessing...
4 rows, 178 columns, 651 non-zeros
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  3.600e+03  ratio =  3.600e+03
GM: min|aij| =  1.560e-01  max|aij| =  6.409e+00  ratio =  4.107e+01
EQ: min|aij| =  2.435e-02  max|aij| =  1.000e+00  ratio =  4.107e+01
Constructing initial basis...
Size of triangular part is 4
      0: obj =   3.863368786e+01 inf =   2.123e+00 (2)
      2: obj =   4.085266937e+01 inf =   0.000e+00 (0)
*     7: obj =   3.498257277e+01 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Time used:   0.0 secs
Memory used: 0.2 Mb (201752 bytes)
Write sensitivity analysis report to 'sensit.sen'...


In [ ]:
# Display the contents of the report: (this shows the sensitivity analysis report)
!cat /content/sensit.sen

GLPK 4.65 - SENSITIVITY ANALYSIS REPORT                                                                         Page   1

Problem:    
Objective:  points = 34.98257277 (MINimum)

   No. Row name     St      Activity         Slack   Lower bound       Activity      Obj coef  Obj value at Limiting
                                          Marginal   Upper bound          range         range   break point variable
------ ------------ -- ------------- ------------- -------------  ------------- ------------- ------------- ------------
     1 c_e_cal_const_
                    NS    1373.64224        .         1373.64224     1345.26973          -Inf      34.16899 onehundredsix
                                            .02867    1373.64224     2231.61879          +Inf      59.58500 fiftyone

     2 c_l_proteinlb_
                    NL      84.00000        .           84.00000       79.99091       -.04461      34.80371 onehundredtwelve
                                            .04461       

In [ ]:
#looping through variable objects in the model and accessing their values

#initializing lists
product_id_list = []
quantity_list = []

for v in model.component_objects(Var, active=True):
    varobject = getattr(model, str(v))
    for index in varobject:
        if varobject[index].value == 0:
          continue
        elif varobject[index].value == None:
          continue  
        else:
#v is a pyomo simplevar type. printing it out then retrieving its contents and input into a list
#also gathering the quantity from the model as well into alist
          old_stdout = sys.stdout 
          sys.stdout = buffer = io.StringIO()
          print(v)
          sys.stdout = old_stdout     
          whatWasPrinted = buffer.getvalue()
          product_id_list.append(whatWasPrinted.replace("\n", ""))
          quantity_list.append(varobject[index].value)
  
product_dict = {'food_drink_id':product_id_list,
                'quantity':quantity_list}

#making a dataframe from dict
answer_df = pd.DataFrame(product_dict)

#merging the original df and the new df for the answer df
answer_df2 = answer_df.merge(df, how='left', on = 'food_drink_id')
answer_df2 = answer_df2[['Food or Drink', 'quantity']]
answer_df2.head()

,Food or Drink,quantity
0,Pastrami Burger,1.366585
1,French Toast Sticks,0.089710
2,1% Low Fat Milk,0.930354
3,"Chocolate Milk, Protein Fortified, Low Fat (1%)",3.471887


# Model 2
This model tries to make for a more realistic diet, including the above restraints from model 1. 

.Liquids are generally worse at sateity than solids, thus we are eliminating liquids. You also have a max of only 1 syrup and 1 sauce. We are also forcing atleast 2 breakfast items a day,  3 snack items (meaning non-liquid, non-dessert food that is less than 300 calories) 1 dessert, and 3 appetizers. Its important to eat food throughout the day to stay satisfied and have nice variety to stay on track with the diet. Also- Diet drinks and 0 calorie items can be drinken but can't be used in the model since they contribute nothing. 

https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2197163/#:~:text=There%20is%20strong%20evidence%20that,return%20of%20hunger%20%5B12%5D.

In [ ]:
from pyomo.environ import *

SOLVER = 'glpk'
EXECUTABLE = '/usr/bin/glpsol'

#initialize model
model = ConcreteModel()

#loop through and create variables
for x in df.values:
  name = x[8]
  model.add_component(name, Var(within=NonNegativeReals))

#Adding these new constraints which will make the end result much more realistic
model.sauce_const = Constraint(expr = model.seventyfive + model.seventysix + model.seventyseven <= 1)

model.syrup_const = Constraint(expr = model.onehundredseven <= 1)

model.dessert = Constraint(expr= model.onehundredfiftyseven + 
model.onehundredfiftyeight + 
model.onehundredfiftynine + 
model.onehundredsixty + 
model.onehundredsixtyone + 
model.onehundredsixtytwo + 
model.onehundredsixtythree + 
model.onehundredsixtyfour + 
model.onehundredsixtyfive + 
model.onehundredsixtysix + 
model.onehundredsixtyseven + 
model.onehundredsixtyeight + 
model.onehundredsixtynine + 
model.onehundredseventy + 
model.onehundredseventyone + 
model.onehundredseventytwo + 
model.onehundredseventythree + 
model.onehundredseventyfour + 
model.onehundredseventyfive + 
model.onehundredseventysix + 
model.onehundredseventyseven + 
model.onehundredseventyeight == 1 )

model.app = Constraint(expr =                        
model.seventyeight + 
model.seventynine + 
model.eighty + 
model.eightyone + 
model.eightytwo + 
model.eightythree + 
model.eightyfour == 3)

model.snack = Constraint(expr = model.onehundredsixtytwo + 
model.onehundredsixtyfour + 
model.onehundredseventysix + 
model.onehundredseventyseven + 
model.onehundredseventyeight + 
model.onehundredseventynine + 
model.one + 
model.two + 
model.three + 
model.four + 
model.five + 
model.six + 
model.seven + 
model.fiftytwo + 
model.sixtyfive + 
model.sixtyseven + 
model.seventyone + 
model.seventyfive + 
model.seventysix + 
model.seventyseven + 
model.seventyeight + 
model.eightythree + 
model.ninetyseven == 3)

model.break_const = Constraint(expr = 
model.eightynine + 
model.ninety + 
model.ninetyone + 
model.ninetytwo + 
model.ninetythree + 
model.ninetyfour + 
model.ninetyfive + 
model.ninetysix + 
model.ninetyseven + 
model.ninetyeight + 
model.ninetynine + 
model.onehundred + 
model.onehundredone + 
model.onehundredtwo + 
model.onehundredthree + 
model.onehundredfour + 
model.onehundredfive + 
model.onehundredsix >= 2)

model.liquid_const = Constraint(expr = model.onehundredeight + 
model.onehundrednine + 
model.onehundredten + 
model.onehundredeleven + 
model.onehundredtwelve + 
model.onehundredthirteen + 
model.onehundredfourteen + 
model.onehundredfifteen + 
model.onehundredsixteen + 
model.onehundredseventeen + 
model.onehundredeighteen + 
model.onehundrednineteen + 
model.onehundredtwenty + 
model.onehundredtwentyone + 
model.onehundredtwentytwo + 
model.onehundredtwentythree + 
model.onehundredtwentyfour + 
model.onehundredtwentyfive + 
model.onehundredtwentysix + 
model.onehundredtwentyseven + 
model.onehundredtwentyeight + 
model.onehundredtwentynine + 
model.onehundredthirty + 
model.onehundredthirtyone + 
model.onehundredthirtytwo + 
model.onehundredthirtythree + 
model.onehundredthirtyfour + 
model.onehundredthirtyfive + 
model.onehundredthirtysix + 
model.onehundredthirtyseven + 
model.onehundredthirtyeight + 
model.onehundredthirtynine + 
model.onehundredforty + 
model.onehundredfortyone + 
model.onehundredfortytwo + 
model.onehundredfortythree + 
model.onehundredfortyfour + 
model.onehundredfortyfive + 
model.onehundredfortysix + 
model.onehundredfortyseven + 
model.onehundredfortyeight + 
model.onehundredfortynine + 
model.onehundredfifty + 
model.onehundredfiftyone + 
model.onehundredfiftytwo + 
model.onehundredfiftythree + 
model.onehundredfiftyfour + 
model.onehundredfiftyfive + 
model.onehundredfiftysix <= 0)

model.cal_const = Constraint(expr = model.zero*320 + 
model.one*270 + 
model.two*240 + 
model.three*250 + 
model.four*240 + 
model.five*250 + 
model.six*240 + 
model.seven*300 + 
model.eight*350 + 
model.nine*340 + 
model.ten*370 + 
model.eleven*460 + 
model.twelve*520 + 
model.thirteen*460 + 
model.fourteen*430 + 
model.fifteen*440 + 
model.sixteen*430 + 
model.seventeen*440 + 
model.eighteen*540 + 
model.nineteen*430 + 
model.twenty*440 + 
model.twentyone*490 + 
model.twentytwo*550 + 
model.twentythree*600 + 
model.twentyfour*370 + 
model.twentyfive*380 + 
model.twentysix*370 + 
model.twentyseven*380 + 
model.twentyeight*510 + 
model.twentynine*370 + 
model.thirty*380 + 
model.thirtyone*430 + 
model.thirtytwo*400 + 
model.thirtythree*450 + 
model.thirtyfour*600 + 
model.thirtyfive*540 + 
model.thirtysix*510 + 
model.thirtyseven*520 + 
model.thirtyeight*510 + 
model.thirtynine*520 + 
model.forty*620 + 
model.fortyone*510 + 
model.fortytwo*520 + 
model.fortythree*570 + 
model.fortyfour*620 + 
model.fortyfive*680 + 
model.fortysix*510 + 
model.fortyseven*610 + 
model.fortyeight*320 + 
model.fortynine*370 + 
model.fifty*640 + 
model.fiftyone*390 + 
model.fiftytwo*300 + 
model.fiftythree*470 + 
model.fiftyfour*350 + 
model.fiftyfive*570 + 
model.fiftysix*380 + 
model.fiftyseven*640 + 
model.fiftyeight*380 + 
model.fiftynine*400 + 
model.sixty*590 + 
model.sixtyone*510 + 
model.sixtytwo*350 + 
model.sixtythree*450 + 
model.sixtyfour*710 + 
model.sixtyfive*250 + 
model.sixtysix*320 + 
model.sixtyseven*280 + 
model.sixtyeight*470 + 
model.sixtynine*490 + 
model.seventy*410 + 
model.seventyone*290 + 
model.seventytwo*360 + 
model.seventythree*540 + 
model.seventyfour*1070 + 
model.seventyfive*150 + 
model.seventysix*60 + 
model.seventyseven*25 + 
model.seventyeight*300 + 
model.seventynine*430 + 
model.eighty*790 + 
model.eightyone*540 + 
model.eightytwo*370 + 
model.eightythree*210 + 
model.eightyfour*420 + 
model.eightyfive*490 + 
model.eightysix*590 + 
model.eightyseven*470 + 
model.eightyeight*370 + 
model.eightynine*330 + 
model.ninety*430 + 
model.ninetyone*390 + 
model.ninetytwo*490 + 
model.ninetythree*440 + 
model.ninetyfour*540 + 
model.ninetyfive*520 + 
model.ninetysix*620 + 
model.ninetyseven*300 + 
model.ninetyeight*400 + 
model.ninetynine*600 + 
model.onehundred*700 + 
model.onehundredone*640 + 
model.onehundredtwo*260 + 
model.onehundredthree*320 + 
model.onehundredfour*350 + 
model.onehundredfive*290 + 
model.onehundredsix*490 + 
model.onehundredseven*120 + 
model.onehundredeight*140 + 
model.onehundrednine*0 + 
model.onehundredten*0 + 
model.onehundredeleven*130 + 
model.onehundredtwelve*110 + 
model.onehundredthirteen*160 + 
model.onehundredfourteen*200 + 
model.onehundredfifteen*150 + 
model.onehundredsixteen*0 + 
model.onehundredseventeen*170 + 
model.onehundredeighteen*140 + 
model.onehundrednineteen*160 + 
model.onehundredtwenty*140 + 
model.onehundredtwentyone*140 + 
model.onehundredtwentytwo*150 + 
model.onehundredtwentythree*200 + 
model.onehundredtwentyfour*0 + 
model.onehundredtwentyfive*230 + 
model.onehundredtwentysix*190 + 
model.onehundredtwentyseven*210 + 
model.onehundredtwentyeight*180 + 
model.onehundredtwentynine*180 + 
model.onehundredthirty*200 + 
model.onehundredthirtyone*250 + 
model.onehundredthirtytwo*0 + 
model.onehundredthirtythree*280 + 
model.onehundredthirtyfour*230 + 
model.onehundredthirtyfive*270 + 
model.onehundredthirtysix*230 + 
model.onehundredthirtyseven*230 + 
model.onehundredthirtyeight*250 + 
model.onehundredthirtynine*320 + 
model.onehundredforty*310 + 
model.onehundredfortyone*370 + 
model.onehundredfortytwo*0 + 
model.onehundredfortythree*420 + 
model.onehundredfortyfour*350 + 
model.onehundredfortyfive*400 + 
model.onehundredfortysix*340 + 
model.onehundredfortyseven*340 + 
model.onehundredfortyeight*375 + 
model.onehundredfortynine*500 + 
model.onehundredfifty*0 + 
model.onehundredfiftyone*570 + 
model.onehundredfiftytwo*470 + 
model.onehundredfiftythree*530 + 
model.onehundredfiftyfour*450 + 
model.onehundredfiftyfive*450 + 
model.onehundredfiftysix*490 + 
model.onehundredfiftyseven*400 + 
model.onehundredfiftyeight*390 + 
model.onehundredfiftynine*370 + 
model.onehundredsixty*400 + 
model.onehundredsixtyone*370 + 
model.onehundredsixtytwo*250 + 
model.onehundredsixtythree*450 + 
model.onehundredsixtyfour*160 + 
model.onehundredsixtyfive*360 + 
model.onehundredsixtysix*820 + 
model.onehundredsixtyseven*650 + 
model.onehundredsixtyeight*650 + 
model.onehundredsixtynine*650 + 
model.onehundredseventy*630 + 
model.onehundredseventyone*630 + 
model.onehundredseventytwo*620 + 
model.onehundredseventythree*630 + 
model.onehundredseventyfour*940 + 
model.onehundredseventyfive*440 + 
model.onehundredseventysix*200 + 
model.onehundredseventyseven*180 + 
model.onehundredseventyeight*170 + 
model.onehundredseventynine*200  == adj_bmr)

model.proteinlb = Constraint(expr = model.zero*15 + 
model.one*12 + 
model.two*10 + 
model.three*10 + 
model.four*10 + 
model.five*10 + 
model.six*10 + 
model.seven*11 + 
model.eight*17 + 
model.nine*12 + 
model.ten*12 + 
model.eleven*19 + 
model.twelve*22 + 
model.thirteen*19 + 
model.fourteen*17 + 
model.fifteen*17 + 
model.sixteen*17 + 
model.seventeen*17 + 
model.eighteen*22 + 
model.nineteen*17 + 
model.twenty*17 + 
model.twentyone*18 + 
model.twentytwo*25 + 
model.twentythree*23 + 
model.twentyfour*13 + 
model.twentyfive*13 + 
model.twentysix*13 + 
model.twentyseven*13 + 
model.twentyeight*17 + 
model.twentynine*13 + 
model.thirty*13 + 
model.thirtyone*14 + 
model.thirtytwo*15 + 
model.thirtythree*18 + 
model.thirtyfour*21 + 
model.thirtyfive*18 + 
model.thirtysix*16 + 
model.thirtyseven*16 + 
model.thirtyeight*16 + 
model.thirtynine*16 + 
model.forty*21 + 
model.fortyone*16 + 
model.fortytwo*16 + 
model.fortythree*17 + 
model.fortyfour*23 + 
model.fortyfive*22 + 
model.fortysix*34 + 
model.fortyseven*39 + 
model.fortyeight*21 + 
model.fortynine*24 + 
model.fifty*43 + 
model.fiftyone*27 + 
model.fiftytwo*19 + 
model.fiftythree*33 + 
model.fiftyfour*22 + 
model.fiftyfive*38 + 
model.fiftysix*24 + 
model.fiftyseven*42 + 
model.fiftyeight*22 + 
model.fiftynine*23 + 
model.sixty*29 + 
model.sixtyone*24 + 
model.sixtytwo*11 + 
model.sixtythree*12 + 
model.sixtyfour*16 + 
model.sixtyfive*7 + 
model.sixtysix*8 + 
model.sixtyseven*12 + 
model.sixtyeight*25 + 
model.sixtynine*22 + 
model.seventy*12 + 
model.seventyone*16 + 
model.seventytwo*23 + 
model.seventythree*34 + 
model.seventyfour*69 + 
model.seventyfive*0 + 
model.seventysix*0 + 
model.seventyseven*0 + 
model.seventyeight*2 + 
model.seventynine*3 + 
model.eighty*6 + 
model.eightyone*12 + 
model.eightytwo*8 + 
model.eightythree*6 + 
model.eightyfour*13 + 
model.eightyfive*24 + 
model.eightysix*25 + 
model.eightyseven*25 + 
model.eightyeight*17 + 
model.eightynine*10 + 
model.ninety*11 + 
model.ninetyone*16 + 
model.ninetytwo*17 + 
model.ninetythree*18 + 
model.ninetyfour*19 + 
model.ninetyfive*19 + 
model.ninetysix*20 + 
model.ninetyseven*16 + 
model.ninetyeight*17 + 
model.ninetynine*20 + 
model.onehundred*21 + 
model.onehundredone*24 + 
model.onehundredtwo*6 + 
model.onehundredthree*12 + 
model.onehundredfour*8 + 
model.onehundredfive*1 + 
model.onehundredsix*6 + 
model.onehundredseven*0 + 
model.onehundredeight*0 + 
model.onehundrednine*0 + 
model.onehundredten*0 + 
model.onehundredeleven*10 + 
model.onehundredtwelve*9 + 
model.onehundredthirteen*8 + 
model.onehundredfourteen*11 + 
model.onehundredfifteen*0 + 
model.onehundredsixteen*0 + 
model.onehundredseventeen*0 + 
model.onehundredeighteen*0 + 
model.onehundrednineteen*0 + 
model.onehundredtwenty*0 + 
model.onehundredtwentyone*0 + 
model.onehundredtwentytwo*0 + 
model.onehundredtwentythree*0 + 
model.onehundredtwentyfour*0 + 
model.onehundredtwentyfive*0 + 
model.onehundredtwentysix*0 + 
model.onehundredtwentyseven*0 + 
model.onehundredtwentyeight*0 + 
model.onehundredtwentynine*0 + 
model.onehundredthirty*0 + 
model.onehundredthirtyone*0 + 
model.onehundredthirtytwo*0 + 
model.onehundredthirtythree*0 + 
model.onehundredthirtyfour*0 + 
model.onehundredthirtyfive*0 + 
model.onehundredthirtysix*0 + 
model.onehundredthirtyseven*0 + 
model.onehundredthirtyeight*0 + 
model.onehundredthirtynine*0 + 
model.onehundredforty*0 + 
model.onehundredfortyone*0 + 
model.onehundredfortytwo*0 + 
model.onehundredfortythree*0 + 
model.onehundredfortyfour*0 + 
model.onehundredfortyfive*0 + 
model.onehundredfortysix*0 + 
model.onehundredfortyseven*0 + 
model.onehundredfortyeight*0 + 
model.onehundredfortynine*0 + 
model.onehundredfifty*0 + 
model.onehundredfiftyone*0 + 
model.onehundredfiftytwo*0 + 
model.onehundredfiftythree*0 + 
model.onehundredfiftyfour*0 + 
model.onehundredfiftyfive*0 + 
model.onehundredfiftysix*0 + 
model.onehundredfiftyseven*7 + 
model.onehundredfiftyeight*7 + 
model.onehundredfiftynine*7 + 
model.onehundredsixty*7 + 
model.onehundredsixtyone*7 + 
model.onehundredsixtytwo*5 + 
model.onehundredsixtythree*6 + 
model.onehundredsixtyfour*3 + 
model.onehundredsixtyfive*4 + 
model.onehundredsixtysix*13 + 
model.onehundredsixtyseven*13 + 
model.onehundredsixtyeight*13 + 
model.onehundredsixtynine*13 + 
model.onehundredseventy*13 + 
model.onehundredseventyone*13 + 
model.onehundredseventytwo*13 + 
model.onehundredseventythree*14 + 
model.onehundredseventyfour*14 + 
model.onehundredseventyfive*7 + 
model.onehundredseventysix*3 + 
model.onehundredseventyseven*3 + 
model.onehundredseventyeight*3 + 
model.onehundredseventynine*3 >= protein_lb)

model.sugar = Constraint(expr = model.zero*4 + 
model.one*4 + 
model.two*4 + 
model.three*6 + 
model.four*5 + 
model.five*6 + 
model.six*4 + 
model.seven*14 + 
model.eight*5 + 
model.nine*4 + 
model.ten*6 + 
model.eleven*4 + 
model.twelve*5 + 
model.thirteen*4 + 
model.fourteen*4 + 
model.fifteen*6 + 
model.sixteen*4 + 
model.seventeen*6 + 
model.eighteen*9 + 
model.nineteen*4 + 
model.twenty*7 + 
model.twentyone*14 + 
model.twentytwo*5 + 
model.twentythree*5 + 
model.twentyfour*7 + 
model.twentyfive*9 + 
model.twentysix*8 + 
model.twentyseven*9 + 
model.twentyeight*12 + 
model.twentynine*7 + 
model.thirty*10 + 
model.thirtyone*17 + 
model.thirtytwo*7 + 
model.thirtythree*7 + 
model.thirtyfour*9 + 
model.thirtyfive*8 + 
model.thirtysix*8 + 
model.thirtyseven*10 + 
model.thirtyeight*8 + 
model.thirtynine*10 + 
model.forty*13 + 
model.fortyone*8 + 
model.fortytwo*11 + 
model.fortythree*18 + 
model.fortyfour*9 + 
model.fortyfive*9 + 
model.fortysix*6 + 
model.fortyseven*6 + 
model.fortyeight*3 + 
model.fortynine*3 + 
model.fifty*4 + 
model.fiftyone*3 + 
model.fiftytwo*5 + 
model.fiftythree*5 + 
model.fiftyfour*5 + 
model.fiftyfive*5 + 
model.fiftysix*5 + 
model.fiftyseven*5 + 
model.fiftyeight*6 + 
model.fiftynine*4 + 
model.sixty*6 + 
model.sixtyone*3 + 
model.sixtytwo*5 + 
model.sixtythree*1 + 
model.sixtyfour*9 + 
model.sixtyfive*1 + 
model.sixtysix*1 + 
model.sixtyseven*6 + 
model.sixtyeight*2 + 
model.sixtynine*4 + 
model.seventy*1 + 
model.seventyone*2 + 
model.seventytwo*1 + 
model.seventythree*2 + 
model.seventyfour*4 + 
model.seventyfive*1 + 
model.seventysix*12 + 
model.seventyseven*2 + 
model.seventyeight*0 + 
model.seventynine*0 + 
model.eighty*18 + 
model.eightyone*1 + 
model.eightytwo*1 + 
model.eightythree*3 + 
model.eightyfour*7 + 
model.eightyfive*3 + 
model.eightysix*3 + 
model.eightyseven*2 + 
model.eightyeight*1 + 
model.eightynine*4 + 
model.ninety*4 + 
model.ninetyone*4 + 
model.ninetytwo*4 + 
model.ninetythree*4 + 
model.ninetyfour*4 + 
model.ninetyfive*8 + 
model.ninetysix*8 + 
model.ninetyseven*1 + 
model.ninetyeight*1 + 
model.ninetynine*1 + 
model.onehundred*1 + 
model.onehundredone*4 + 
model.onehundredtwo*4 + 
model.onehundredthree*4 + 
model.onehundredfour*4 + 
model.onehundredfive*0 + 
model.onehundredsix*11 + 
model.onehundredseven*21 + 
model.onehundredeight*28 + 
model.onehundrednine*0 + 
model.onehundredten*0 + 
model.onehundredeleven*13 + 
model.onehundredtwelve*12 + 
model.onehundredthirteen*25 + 
model.onehundredfourteen*33 + 
model.onehundredfifteen*41 + 
model.onehundredsixteen*0 + 
model.onehundredseventeen*46 + 
model.onehundredeighteen*39 + 
model.onehundrednineteen*43 + 
model.onehundredtwenty*39 + 
model.onehundredtwentyone*35 + 
model.onehundredtwentytwo*41 + 
model.onehundredtwentythree*55 + 
model.onehundredtwentyfour*0 + 
model.onehundredtwentyfive*61 + 
model.onehundredtwentysix*52 + 
model.onehundredtwentyseven*57 + 
model.onehundredtwentyeight*52 + 
model.onehundredtwentynine*45 + 
model.onehundredthirty*54 + 
model.onehundredthirtyone*68 + 
model.onehundredthirtytwo*0 + 
model.onehundredthirtythree*77 + 
model.onehundredthirtyfour*65 + 
model.onehundredthirtyfive*72 + 
model.onehundredthirtysix*65 + 
model.onehundredthirtyseven*58 + 
model.onehundredthirtyeight*68 + 
model.onehundredthirtynine*84 + 
model.onehundredforty*81 + 
model.onehundredfortyone*103 + 
model.onehundredfortytwo*0 + 
model.onehundredfortythree*115 + 
model.onehundredfortyfour*97 + 
model.onehundredfortyfive*107 + 
model.onehundredfortysix*97 + 
model.onehundredfortyseven*86 + 
model.onehundredfortyeight*101 + 
model.onehundredfortynine*137 + 
model.onehundredfifty*0 + 
model.onehundredfiftyone*153 + 
model.onehundredfiftytwo*130 + 
model.onehundredfiftythree*143 + 
model.onehundredfiftyfour*130 + 
model.onehundredfiftyfive*115 + 
model.onehundredfiftysix*133 + 
model.onehundredfiftyseven*50 + 
model.onehundredfiftyeight*51 + 
model.onehundredfiftynine*48 + 
model.onehundredsixty*53 + 
model.onehundredsixtyone*49 + 
model.onehundredsixtytwo*30 + 
model.onehundredsixtythree*37 + 
model.onehundredsixtyfour*18 + 
model.onehundredsixtyfive*25 + 
model.onehundredsixtysix*122 + 
model.onehundredsixtyseven*91 + 
model.onehundredsixtyeight*91 + 
model.onehundredsixtynine*92 + 
model.onehundredseventy*76 + 
model.onehundredseventyone*80 + 
model.onehundredseventytwo*78 + 
model.onehundredseventythree*78 + 
model.onehundredseventyfour*87 + 
model.onehundredseventyfive*73 + 
model.onehundredseventysix*26 + 
model.onehundredseventyseven*28 + 
model.onehundredseventyeight*25 + 
model.onehundredseventynine*30 <= sugar_cal_b)

model.sodium = Constraint(expr = model.zero*1410 + 
model.one*1160 + 
model.two*810 + 
model.three*870 + 
model.four*1040 + 
model.five*1230 + 
model.six*760 + 
model.seven*2060 + 
model.eight*1670 + 
model.nine*880 + 
model.ten*970 + 
model.eleven*1720 + 
model.twelve*1440 + 
model.thirteen*1350 + 
model.fourteen*1000 + 
model.fifteen*1060 + 
model.sixteen*1230 + 
model.seventeen*1420 + 
model.eighteen*1310 + 
model.nineteen*950 + 
model.twenty*1100 + 
model.twentyone*2250 + 
model.twentytwo*1720 + 
model.twentythree*1800 + 
model.twentyfour*880 + 
model.twentyfive*930 + 
model.twentysix*1100 + 
model.twentyseven*1300 + 
model.twentyeight*1170 + 
model.twentynine*830 + 
model.thirty*970 + 
model.thirtyone*2120 + 
model.thirtytwo*1220 + 
model.thirtythree*1480 + 
model.thirtyfour*1420 + 
model.thirtyfive*1330 + 
model.thirtysix*990 + 
model.thirtyseven*1040 + 
model.thirtyeight*1210 + 
model.thirtynine*1410 + 
model.forty*1300 + 
model.fortyone*940 + 
model.fortytwo*1080 + 
model.fortythree*2230 + 
model.fortyfour*1850 + 
model.fortyfive*1780 + 
model.fortysix*870 + 
model.fortyseven*1380 + 
model.fortyeight*870 + 
model.fortynine*1130 + 
model.fifty*1940 + 
model.fiftyone*980 + 
model.fiftytwo*650 + 
model.fiftythree*810 + 
model.fiftyfour*910 + 
model.fiftyfive*1320 + 
model.fiftysix*1020 + 
model.fiftyseven*1550 + 
model.fiftyeight*970 + 
model.fiftynine*1480 + 
model.sixty*1810 + 
model.sixtyone*1400 + 
model.sixtytwo*640 + 
model.sixtythree*860 + 
model.sixtyfour*1820 + 
model.sixtyfive*490 + 
model.sixtysix*540 + 
model.sixtyseven*1180 + 
model.sixtyeight*1880 + 
model.sixtynine*1870 + 
model.seventy*1200 + 
model.seventyone*690 + 
model.seventytwo*1100 + 
model.seventythree*1640 + 
model.seventyfour*3290 + 
model.seventyfive*140 + 
model.seventysix*220 + 
model.seventyseven*1590 + 
model.seventyeight*400 + 
model.seventynine*630 + 
model.eighty*3600 + 
model.eightyone*1380 + 
model.eightytwo*800 + 
model.eightythree*670 + 
model.eightyfour*1340 + 
model.eightyfive*1380 + 
model.eightysix*1500 + 
model.eightyseven*1720 + 
model.eightyeight*1180 + 
model.eightynine*1120 + 
model.ninety*1240 + 
model.ninetyone*1270 + 
model.ninetytwo*1390 + 
model.ninetythree*1520 + 
model.ninetyfour*1640 + 
model.ninetyfive*990 + 
model.ninetysix*1110 + 
model.ninetyseven*830 + 
model.ninetyeight*950 + 
model.ninetynine*960 + 
model.onehundred*1080 + 
model.onehundredone*1820 + 
model.onehundredtwo*890 + 
model.onehundredthree*1040 + 
model.onehundredfour*1180 + 
model.onehundredfive*240 + 
model.onehundredsix*460 + 
model.onehundredseven*25 + 
model.onehundredeight*0 + 
model.onehundrednine*0 + 
model.onehundredten*0 + 
model.onehundredeleven*130 + 
model.onehundredtwelve*130 + 
model.onehundredthirteen*210 + 
model.onehundredfourteen*230 + 
model.onehundredfifteen*35 + 
model.onehundredsixteen*35 + 
model.onehundredseventeen*70 + 
model.onehundredeighteen*35 + 
model.onehundrednineteen*65 + 
model.onehundredtwenty*45 + 
model.onehundredtwentyone*75 + 
model.onehundredtwentytwo*160 + 
model.onehundredtwentythree*45 + 
model.onehundredtwentyfour*45 + 
model.onehundredtwentyfive*95 + 
model.onehundredtwentysix*45 + 
model.onehundredtwentyseven*85 + 
model.onehundredtwentyeight*60 + 
model.onehundredtwentynine*100 + 
model.onehundredthirty*210 + 
model.onehundredthirtyone*60 + 
model.onehundredthirtytwo*60 + 
model.onehundredthirtythree*115 + 
model.onehundredthirtyfour*60 + 
model.onehundredthirtyfive*110 + 
model.onehundredthirtysix*75 + 
model.onehundredthirtyseven*125 + 
model.onehundredthirtyeight*260 + 
model.onehundredthirtynine*250 + 
model.onehundredforty*260 + 
model.onehundredfortyone*90 + 
model.onehundredfortytwo*90 + 
model.onehundredfortythree*170 + 
model.onehundredfortyfour*85 + 
model.onehundredfortyfive*160 + 
model.onehundredfortysix*110 + 
model.onehundredfortyseven*190 + 
model.onehundredfortyeight*390 + 
model.onehundredfortynine*115 + 
model.onehundredfifty*115 + 
model.onehundredfiftyone*230 + 
model.onehundredfiftytwo*115 + 
model.onehundredfiftythree*220 + 
model.onehundredfiftyfour*150 + 
model.onehundredfiftyfive*250 + 
model.onehundredfiftysix*520 + 
model.onehundredfiftyseven*250 + 
model.onehundredfiftyeight*250 + 
model.onehundredfiftynine*220 + 
model.onehundredsixty*250 + 
model.onehundredsixtyone*230 + 
model.onehundredsixtytwo*180 + 
model.onehundredsixtythree*200 + 
model.onehundredsixtyfour*115 + 
model.onehundredsixtyfive*135 + 
model.onehundredsixtysix*340 + 
model.onehundredsixtyseven*440 + 
model.onehundredsixtyeight*480 + 
model.onehundredsixtynine*440 + 
model.onehundredseventy*410 + 
model.onehundredseventyone*420 + 
model.onehundredseventytwo*440 + 
model.onehundredseventythree*460 + 
model.onehundredseventyfour*540 + 
model.onehundredseventyfive*290 + 
model.onehundredseventysix*130 + 
model.onehundredseventyseven*135 + 
model.onehundredseventyeight*105 + 
model.onehundredseventynine*135 <= sodium_b)

In [ ]:
from pyomo.environ import *

#objective function
model.points = Objective(
    expr = model.zero*9 + 
model.one*8 + 
model.two*7 + 
model.three*7 + 
model.four*7 + 
model.five*7 + 
model.six*7 + 
model.seven*8 + 
model.eight*10 + 
model.nine*9 + 
model.ten*10 + 
model.eleven*13 + 
model.twelve*14 + 
model.thirteen*12 + 
model.fourteen*12 + 
model.fifteen*12 + 
model.sixteen*12 + 
model.seventeen*12 + 
model.eighteen*15 + 
model.nineteen*12 + 
model.twenty*12 + 
model.twentyone*14 + 
model.twentytwo*15 + 
model.twentythree*17 + 
model.twentyfour*10 + 
model.twentyfive*11 + 
model.twentysix*10 + 
model.twentyseven*11 + 
model.twentyeight*14 + 
model.twentynine*10 + 
model.thirty*11 + 
model.thirtyone*12 + 
model.thirtytwo*11 + 
model.thirtythree*13 + 
model.thirtyfour*16 + 
model.thirtyfive*15 + 
model.thirtysix*14 + 
model.thirtyseven*14 + 
model.thirtyeight*14 + 
model.thirtynine*14 + 
model.forty*17 + 
model.fortyone*14 + 
model.fortytwo*14 + 
model.fortythree*16 + 
model.fortyfour*17 + 
model.fortyfive*19 + 
model.fortysix*15 + 
model.fortyseven*18 + 
model.fortyeight*9 + 
model.fortynine*10 + 
model.fifty*19 + 
model.fiftyone*10 + 
model.fiftytwo*8 + 
model.fiftythree*14 + 
model.fiftyfour*9 + 
model.fiftyfive*17 + 
model.fiftysix*10 + 
model.fiftyseven*19 + 
model.fiftyeight*10 + 
model.fiftynine*10 + 
model.sixty*16 + 
model.sixtyone*14 + 
model.sixtytwo*10 + 
model.sixtythree*12 + 
model.sixtyfour*20 + 
model.sixtyfive*7 + 
model.sixtysix*9 + 
model.sixtyseven*7 + 
model.sixtyeight*12 + 
model.sixtynine*14 + 
model.seventy*11 + 
model.seventyone*8 + 
model.seventytwo*10 + 
model.seventythree*14 + 
model.seventyfour*29 + 
model.seventyfive*4 + 
model.seventysix*1 + 
model.seventyseven*1 + 
model.seventyeight*9 + 
model.seventynine*12 + 
model.eighty*22 + 
model.eightyone*15 + 
model.eightytwo*11 + 
model.eightythree*6 + 
model.eightyfour*11 + 
model.eightyfive*13 + 
model.eightysix*16 + 
model.eightyseven*12 + 
model.eightyeight*10 + 
model.eightynine*9 + 
model.ninety*12 + 
model.ninetyone*11 + 
model.ninetytwo*14 + 
model.ninetythree*12 + 
model.ninetyfour*15 + 
model.ninetyfive*14 + 
model.ninetysix*17 + 
model.ninetyseven*8 + 
model.ninetyeight*11 + 
model.ninetynine*17 + 
model.onehundred*19 + 
model.onehundredone*18 + 
model.onehundredtwo*7 + 
model.onehundredthree*9 + 
model.onehundredfour*10 + 
model.onehundredfive*8 + 
model.onehundredsix*13 + 
model.onehundredseven*3 + 
model.onehundredeight*4 + 
model.onehundrednine*0 + 
model.onehundredten*0 + 
model.onehundredeleven*4 + 
model.onehundredtwelve*3 + 
model.onehundredthirteen*4 + 
model.onehundredfourteen*5 + 
model.onehundredfifteen*4 + 
model.onehundredsixteen*0 + 
model.onehundredseventeen*5 + 
model.onehundredeighteen*4 + 
model.onehundrednineteen*5 + 
model.onehundredtwenty*4 + 
model.onehundredtwentyone*4 + 
model.onehundredtwentytwo*4 + 
model.onehundredtwentythree*6 + 
model.onehundredtwentyfour*0 + 
model.onehundredtwentyfive*7 + 
model.onehundredtwentysix*6 + 
model.onehundredtwentyseven*6 + 
model.onehundredtwentyeight*6 + 
model.onehundredtwentynine*5 + 
model.onehundredthirty*6 + 
model.onehundredthirtyone*7 + 
model.onehundredthirtytwo*0 + 
model.onehundredthirtythree*8 + 
model.onehundredthirtyfour*7 + 
model.onehundredthirtyfive*8 + 
model.onehundredthirtysix*7 + 
model.onehundredthirtyseven*6 + 
model.onehundredthirtyeight*7 + 
model.onehundredthirtynine*9 + 
model.onehundredforty*9 + 
model.onehundredfortyone*11 + 
model.onehundredfortytwo*0 + 
model.onehundredfortythree*12 + 
model.onehundredfortyfour*11 + 
model.onehundredfortyfive*12 + 
model.onehundredfortysix*11 + 
model.onehundredfortyseven*9 + 
model.onehundredfortyeight*11 + 
model.onehundredfortynine*15 + 
model.onehundredfifty*0 + 
model.onehundredfiftyone*17 + 
model.onehundredfiftytwo*14 + 
model.onehundredfiftythree*16 + 
model.onehundredfiftyfour*14 + 
model.onehundredfiftyfive*12 + 
model.onehundredfiftysix*14 + 
model.onehundredfiftyseven*12 + 
model.onehundredfiftyeight*11 + 
model.onehundredfiftynine*11 + 
model.onehundredsixty*11 + 
model.onehundredsixtyone*11 + 
model.onehundredsixtytwo*7 + 
model.onehundredsixtythree*13 + 
model.onehundredsixtyfour*5 + 
model.onehundredsixtyfive*10 + 
model.onehundredsixtysix*24 + 
model.onehundredsixtyseven*19 + 
model.onehundredsixtyeight*19 + 
model.onehundredsixtynine*19 + 
model.onehundredseventy*18 + 
model.onehundredseventyone*18 + 
model.onehundredseventytwo*18 + 
model.onehundredseventythree*19 + 
model.onehundredseventyfour*28 + 
model.onehundredseventyfive*13 + 
model.onehundredseventysix*6 + 
model.onehundredseventyseven*5 + 
model.onehundredseventyeight*5 + 
model.onehundredseventynine*6, sense=minimize
    )

#took out the .write to get more room on my screen
SolverFactory(SOLVER, executable=EXECUTABLE).solve(model)

{'Problem': [{'Name': 'unknown', 'Lower bound': -inf, 'Upper bound': inf, 'Number of objectives': 1, 'Number of constraints': 12, 'Number of variables': 181, 'Number of nonzeros': 775, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'other', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}}, 'Error rc': 0, 'Time': 0.012289047241210938}]}

## Model 2 Results

Here, we output the models results and the weight watchers points. It is common to NOT get a result as the model is unable to work; we have a strict sodium limit as per the CDC guidelines and the menu is high in salt. Thus, higher calorie diets typically can't work. This is especially true when we make the diet more realistic and require dinner/breakfast items.

In [ ]:
#printing model results and weight watchers points
SolverFactory(SOLVER, executable=EXECUTABLE).solve(model).write()
print(" ")
print("Weight Watchers Points: " + str(model.points()))

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 12
  Number of variables: 181
  Number of nonzeros: 775
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: other
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.011943578720092773
 
ERROR: evaluating object as numeric value: zero
        (object: <class 'pyomo.core.base.var.SimpleVar'>)
    No value for

ValueError: ignored

In [ ]:
# Save the model 
model.write("/content/model.lp", io_options={'symbolic_solver_labels': True})

# Generate the file "sensit.sen", this contains the report we want to see
!/usr/bin/glpsol -m /content/model.lp --lp --ranges sensit.sen

In [ ]:
# Display the contents of the report: (this shows the sensitivity analysis report)
!cat /content/sensit.sen

In [ ]:
#looping through variable objects in the model and accessing their values

#initializing lists
product_id_list = []
quantity_list = []

for v in model.component_objects(Var, active=True):
    varobject = getattr(model, str(v))
    for index in varobject:
        if varobject[index].value == 0:
          continue
        elif varobject[index].value == None:
          continue  
        else:
#v is a pyomo simplevar type. printing it out then retrieving its contents and input into a list
#also gathering the quantity from the model as well into alist
          old_stdout = sys.stdout 
          sys.stdout = buffer = io.StringIO()
          print(v)
          sys.stdout = old_stdout     
          whatWasPrinted = buffer.getvalue()
          product_id_list.append(whatWasPrinted.replace("\n", ""))
          quantity_list.append(varobject[index].value)
  
product_dict = {'food_drink_id':product_id_list,
                'quantity':quantity_list}

#making a dataframe from dict
answer_df = pd.DataFrame(product_dict)

#merging the original df and the new df for the answer df
answer_df2 = answer_df.merge(df, how='inner', on = 'food_drink_id')
answer_df2 = answer_df2[['Food or Drink', 'quantity']]
answer_df2.head(100)

# Conclusion

There is a huge difference between the two models. The second model is much more realistic as it allows for an actual dinner/breakfast and limits rediculous items like syrup. This allows for a more controlled chaos instead of the amalganation of random offerings from the first model. Whats great is that we didn't make our diet unhealthier with these adjustments aswell. Furthermore, we learned that not everyone can benefit from eating only Wienersschniztel whilst doing our Healthy-IIFYM due to the sodium constraint; sometimes its impossible to balance the protein and sodium constraint! 

Overall, we were pleased with our blended IIFYM + CDC Health Guidlines as it allows for one to still be healthy and lose weight with this diet!

# Learnings

*   Diets don't have to be insanely restrictive like cutting out carbs as long as they are data driven! With our diet, they are free to eat whatever they wanted as long as its within their calorie goals and their gurenteed to lose weight!
*   Wienerschniztal (and fast food) is insanely unhealthy! Our model failed plenty of times.
*   We should pay attention to "micros" as well! Things like sodium are important to keep an eye on to maintain good health; not many diets in our research payed attention to micros
*I would suggest inputting all fast food menus alongside a column indicating the fast food place. Run the model and output the resulting answer dataframe; would be interesting to see which fast food place has the best items
*Perhaps eliminate usage of Weight Watchers points and instead focus on minimizing price; realistically, people take price into consideration as well for their purchases



In [ ]:
model.break_const = Constraint(expr = 
model.eightynine + 
model.ninety + 
model.ninetyone + 
model.ninetytwo + 
model.ninetythree + 
model.ninetyfour + 
model.ninetyfive + 
model.ninetysix + 
model.ninetyseven + 
model.ninetyeight + 
model.ninetynine + 
model.onehundred + 
model.onehundredone + 
model.onehundredtwo + 
model.onehundredthree + 
model.onehundredfour + 
model.onehundredfive + 
model.onehundredsix >= 1)

model.din_const = Constraint(expr = 
model.zero +
model.one + 
model.two + 
model.three + 
model.four + 
model.five + 
model.six + 
model.seven + 
model.eight + 
model.nine + 
model.ten + 
model.eleven + 
model.twelve + 
model.thirteen + 
model.fourteen + 
model.fifteen + 
model.sixteen + 
model.seventeen + 
model.eighteen + 
model.nineteen + 
model.twenty + 
model.twentyone + 
model.twentytwo + 
model.twentythree + 
model.twentyfour + 
model.twentyfive + 
model.twentysix + 
model.twentyseven + 
model.twentyeight + 
model.twentynine + 
model.thirty + 
model.thirtyone + 
model.thirtytwo + 
model.thirtythree + 
model.thirtyfour + 
model.thirtyfive + 
model.thirtysix + 
model.thirtyseven + 
model.thirtyeight + 
model.thirtynine + 
model.forty + 
model.fortyone + 
model.fortytwo + 
model.fortythree + 
model.fortyfour + 
model.fortyfive + 
model.fortysix + 
model.fortyseven + 
model.fortyeight + 
model.fortynine + 
model.fifty + 
model.fiftyone + 
model.fiftytwo + 
model.fiftythree + 
model.fiftyfour + 
model.fiftyfive + 
model.fiftysix + 
model.fiftyseven + 
model.fiftyeight + 
model.fiftynine + 
model.sixty + 
model.sixtyone + 
model.sixtytwo + 
model.sixtythree + 
model.sixtyfour + 
model.sixtyfive + 
model.sixtysix + 
model.sixtyseven + 
model.sixtyeight + 
model.sixtynine + 
model.seventy + 
model.seventyone + 
model.seventytwo + 
model.seventythree + 
model.seventyfour >= 1)

#dataframe of just foods, no liquid
#all liquids are 108+ so its an easy cut
new_df = df.copy()
new_df['row_id'] = df.index
new_df = new_df[(new_df.row_id >= 157)]
#new_df2 = new_df[(new_df.row_id <= 107)]

#new_df = pd.concat([new_df1, new_df2])

#new_df = new_df[(new_df.Calories <= 300)]

len(new_df)

#Loop we used to automate the writing of the hard code for the constraint
#borrowing aboves loop to rewrite constraint; below is an example

constraint_list = list()
for x in new_df.values:
  hard_coded_constraint = "model." + str(x[8]) + " + " 
  constraint_list.append(hard_coded_constraint)

for x in constraint_list:
    con = x.replace("'", "")
    print(con)
    #break